# Property Based Testing

## You shall not pass!



# About me and Property based tsting

- My name is Yogesh Sajanikar. I am a Haskeller and an author. A former member of Consensus Algorithm team for Cardano Blockchain, where we used QuickCheck, the first library written to help with property based testing.
- I was fortunate to learn `QuickCheck` directly from [John Hughes](https://en.wikipedia.org/wiki/John_Hughes_(computer_scientist)) who is an inventor of Property based testing

# Testing 

<p style="text-align: center"> <em>Testing is the most commonly used approach for assuring software quality </em></p>

### Types of Tests

* Manual tests
* Unit tests
* Integration tests
* System tests


# Cost of Software Quality Assurance

Upto 50% cost of the software is consumed by the testing activity. The cost of the software quality is borne by

- the developer,

- the QA

- the customer!

# Time vs cost of quality

![Time and cost of quality](https://learn.microsoft.com/en-us/previous-versions/visualstudio/cross-platform/tools-for-cordova/debug-test/media/primer/03-bug-cost-over-time.png?view=toolsforcordova-2017)

[Microsoft: Unit testing and continuous integration](https://learn.microsoft.com/en-us/previous-versions/visualstudio/cross-platform/tools-for-cordova/debug-test/unit-test-primer?view=toolsforcordova-2017)

# Property based Testing
## The idea!

- Tell if the test has passed or failed! (Obvious)
- Should be able to _generate_ tests (Ok!)
- If it can zoom into the bug (Nice!)

## History
- [K. Classen and John Hughes](https://www.cs.tufts.edu/~nr/cs257/archive/john-hughes/quick.pdf) published the paper about `QuickCheck`, a Haskell based library for property based testing.
- QuickCheck also appeared in Erlang.
- Ported to almost all major languages including C, C++, Scala, Rust, Java, Python ...

## Code in this presentation
- Uses `Python` as the language
- Uses `hypothesis` as the property based testing framework

# Get to work


In [ ]:
import ipytest
ipytest.autoconfig()

In [ ]:
import numpy as np
from hypothesis import strategies as st, given, target
from typing import List, TypeVar

T = TypeVar('T')

In [ ]:
def find_max(inp: List[T]) -> T:
    max = -1 #inp[0] 
    for i in inp:
        if i > max:
            max = i
    return max

In [ ]:
%%ipytest -v

def test_find_max():
    inp = [i for i in range(0, 10)]
    mx = find_max(inp)
    assert mx == 9

In [ ]:
%%ipytest -q

@given(st.lists(st.integers(), min_size=1, ))
def test_prop_find_max(inp: List[T]):
    inp_copy = inp[:]
    inp.sort(reverse=True)
    assert inp[0] == find_max(inp_copy)

## Run again with some insight

In [ ]:
from hypothesis import find, settings, Verbosity

In [ ]:
@given(st.lists(st.integers(), min_size=1))
@settings(verbosity=Verbosity.verbose)
def test_prop_find_max(inp: List[T]):
    inp_copy = inp[:]
    inp.sort(reverse=True)
    assert inp[0] == find_max(inp_copy)

In [ ]:
test_prop_find_max()

## Another example - Reverse a list

In [ ]:
def reverse_list(inp: List[T]) -> List[T]:
    retval = inp[:]
    n = len(inp)
    for i in range(n//2):
        retval[i], retval[-i-1] = inp[-i-1], inp[i]
    
    return retval

In [ ]:
#%%ipytest --hypothesis-verbosity=verbose  --hypothesis-show-statistics

@given(st.lists(st.integers()))
@settings(verbosity=Verbosity.verbose)
def test_reverse_list(inp: List[int]):
    assert reverse_list(reverse_list(inp)) == inp

In [ ]:
test_reverse_list()

# Property Testing Internals

* Generators - Well distributed *valid* random data
* A property that holds **true** under valid input
* An ability to *zoom* on to the issue within set limits

## Random Data Generation

- Generating valid data 
- Limiting the number of examples `@settings(max_examples=500)`

### Valid data generation
- Generating valid data is essential to a test. 
- Preconditioning the data by creating a custom strategy.

In [ ]:
# Generate a tuple
st.tuples(st.integers(), st.integers()).example()

In [ ]:
# Filter the tuple 
st.tuples(st.integers(), st.integers()).filter(lambda xy: xy[0] > xy[1]).example()

## Generating valid input (Composite strategy)

In [ ]:
# Create a composite strategy
@st.composite
def monotonic_tuple(draw, types=st.integers()):
    x = draw(st.integers())
    y = draw(st.integers(min_value=x+1))
    return y, x

In [ ]:
monotonic_tuple().example()

## Targeting the problem areas

We will not get into the details of shrinking the data.

- Create a UV that needs to be minimized or optimized
- Uses _Hill Climbing_ or _Simulated Annealing_ to find the problem areas
- The **distance** from the _property_ is usually maximized/minimized.
- This is also called _**target**_ as it is used for finding issues.

## Targeted Property Test

In [ ]:
# The target can also be specified within the test

@given(st.floats(0, 1e100), st.floats(0, 1e100), st.floats(0, 1e100))
@settings(verbosity=Verbosity.verbose)
def test_associativity_with_target(a, b, c):
    ab_c = (a + b) + c
    a_bc = a + (b + c)
    difference = abs(ab_c - a_bc)
    target(difference)  # Without this, the test almost always passes
    assert difference < 2.0

In [ ]:
test_associativity_with_target()

# Why do I care about properties

## Case Study - Mnesia

[Model based testing of data constraints: Testing the business logic of a mnesia application with Quviq QuickCheck](https://www.researchgate.net/publication/221211415_Model_based_testing_of_data_constraints_Testing_the_business_logic_of_a_mnesia_application_with_Quviq_QuickCheck)

- Documents how property based testing helped overcome infrequent but regular crashes that caused *KRED* application engineers to restart *Mnesia* database. 
- This was done by creating a SQL query that would always remain invariant.

## Case Study - AutoSAR 
- AutoSAR is an automotive open architecture for electronic systems
- AutoSAR specs evolved over years, and progressed from 8bit to 16 bit to newer processors
- The essential systems (braking) were mapped to lower addresses, whereas systems such as *music* mounted on higher addresses. The essential systems are always prioritized in the decreasing order of addresses. Due to newer system mounted on higher addresses, the _accessories_ would get priority in the new standard.
- AutoSAR modified its standard to correct this _mistake_. 

## Thinking about properties

- Thinking about invariants in a system is **hard**
- Examples 
  - The *sum* during a transaction should be **constant**
  - GET after a POST/DELETE should always be **None** (Or without 500/400 as desired).
  - A _stateful_ system can speficy a **FSM** as a valid state
  - PDF creation service should result in certain **tags** for all entries in the database. 

## Properties as _Specifications_

- Properties represent *test specifications* of the system. 
- They are derived from the *specifications* of the system
- With property based *test specifications* one has to be more specific than just being objective. 
  - How the the input is structured?
  - Business rules need to be quantified?
  - At the implementation level, the specification *should* be split appropriately. 
    - What guarantee is given by SQL vs logic maintained in the code!
  

This process itself _affects_ the design, and structure of the project.